In [7]:
from gensim.models import KeyedVectors
filename = "./GoogleNews-vectors-negative300.bin"
model = KeyedVectors.load_word2vec_format(filename, binary = True)

In [1]:
### STANFORD WORD2VEC ###
#create word2vec model:

from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'glove.6B.50d.txt'
word2vec_output_file = 'word2vec.txt'
glove2word2vec(glove_input_file, word2vec_output_file)

(400000, 50)

In [3]:
#create word2vec model: 

from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('word2vec.txt', binary = False)

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

money = pd.read_csv("./final_csvs/money.csv")
text = money["text"].str.strip("['[").str.strip("']") #.str.strip("By w+,")

In [4]:
result = model.most_similar(positive = ['woman', 'king'], negative = ['man'], topn=1)

In [5]:
result

[('queen', 0.8523603677749634)]